# Production of dredging vessels

Trailing Suction Hopper Dredgers (TSHDs) and Water Injection Dredgers (WIDs) are the main vessels used for maintenance dredging in ports and waterways. The efficiency of reallocating/remobilizing sediments by these vessels can be measured by determining the production of these vessels which is the amount of dredged sediments per period of time.

In this notebook, a basic simulation is set up to determine the production of TSHDs and WIDs in a simple network. Some properties such as the location of dredging and discharging are added to the project network along with vessel properties.

The following steps are taken to conduct the simulation:

* Import libraries
* Initialise simpy environment
* Define object classes
* Create objects
  * Create sites
  * Create vessels
  * Create activities
* Register processes and run simpy

### 0. Importing libraries
It starts with importing libraries

In [30]:
#packages related to time, space, and id
import datetime, time
import math
import haversine as hs

# package(s) related to the simulation (creating the vessel, running the simulation)
import simpy
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugin

# package(s) needed for inspecting the output
import pandas as pd

# package(s) needed for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# package(s) used for creating and geo-locating the graph (spatial libraries)  
import shapely.geometry
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

### 1. Initialize simpy Environment

In [31]:
# setup environment (simulation time needs to match the project starting time)
simulation_start  = datetime.datetime(2022, 1, 1, 0, 0)
my_env = simpy.Environment(initial_time=simulation_start.timestamp())

### 2. Define Object classes

In [32]:
# create a site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource
    ),
    {})


# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction
    ),
    {})

### 3. Create Objects

#### 3.1. Create site objects

In [33]:
# prepare input data for from_site
location_from_site = shapely.geometry.Point(1.1950, 104.1036)
data_from_site = {"env": my_env,
                  "name": "from_site",
                  "geometry": location_from_site,
                  "capacity": 10_000,
                  "level": 10_000
                 }
# instantiate from_site 
from_site = Site(**data_from_site)

# prepare input data for to_site
location_to_site = shapely.geometry.Point(1.2131, 104.1434)
data_to_site = {"env": my_env,
                "name": "to_site",
                "geometry": location_to_site,
                "capacity": 10_000,
                "level": 0
               }
# instantiate to_site 
to_site = Site(**data_to_site)

#### 3.2. Create TSHD object(s)

In [39]:
# prepare input data for sagarmanthan (WID), ham311 (TSHD), and asia(TSHD)
data_asia = {"env": my_env,
                     "name": "asia",
                     "geometry": location_from_site,
                     "loading_rate": 1,
                     "unloading_rate": 5,
                     "capacity": 1000,
                     "compute_v": lambda x: 16 + 2 * x}

# instantiate asia
asia = TransportProcessingResource(**data_asia)


class VesselProperties:
    
    """
    type: can contain info on vessel type (avv class, cemt_class or other)
    B: vessel width
    L: vessel length
    H_e: vessel height unloaded
    H_f: vessel height loaded
    T_e: draught unloaded
    T_f: draught loaded

    Add information on possible restrictions to the vessels, i.e. height, width, etc.
    """

    def __init__(
        self,
        type,
        B,
        L,
        H_e,
        H_f,
        T_e,
        T_f,
        *args,
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.type = type

        self.B = B
        self.L = L

        self.H_e = H_e
        self.H_f = H_e

        self.T_e = T_e
        self.T_f = T_f


        
    @property
    def H(self):
        """ Calculate current height based on filling degree """

        return (
            self.filling_degree * (self.H_f - self.H_e)
            + self.H_e
        )

    @property
    def T(self):
        """ Calculate current draught based on filling degree
        
        Here we should implement the rules from Van Dorsser et al 
        https://www.researchgate.net/publication/344340126_The_effect_of_low_water_on_loading_capacity_of_inland_ships
        """

        return (
            self.filling_degree * (self.T_f - self.T_e)
            + self.T_e
        )

    def get_route(
        self,
        origin,
        destination,
        graph=None,
        minWidth=None,
        minHeight=None,
        minDepth=None,
        randomSeed=4,
    ):
        """ Calculate a path based on vessel restrictions """

        graph = graph if graph else self.env.FG
        minWidth = minWidth if minWidth else 1.1 * self.B
        minHeight = minWidth if minHeight else 1.1 * self.H
        minDepth = minWidth if minDepth else 1.1 * self.T

        # Check if information on restrictions is added to the edges
        random.seed(randomSeed)
        edge = random.choice(list(graph.edges(data=True)))
        edge_attrs = list(edge[2].keys())

        # IMPROVE THIS TO CHECK ALL EDGES AND COMBINATIONS OF RESTRICTIONS

        if all(item in edge_attrs for item in ["Width", "Height", "Depth"]):
            edges = []
            nodes = []

            for edge in graph.edges(data=True):
                if (
                    edge[2]["Width"] >= minWidth
                    and edge[2]["Height"] >= minHeight
                    and edge[2]["Depth"] >= minDepth
                ):
                    edges.append(edge)

                    nodes.append(graph.nodes[edge[0]])
                    nodes.append(graph.nodes[edge[1]])

            subGraph = graph.__class__()

            for node in nodes:
                subGraph.add_node(
                    node["name"],
                    name=node["name"],
                    geometry=node["geometry"],
                    position=(node["geometry"].x, node["geometry"].y),
                )

            for edge in edges:
                subGraph.add_edge(edge[0], edge[1], attr_dict=edge[2])

            try:
                return nx.dijkstra_path(subGraph, origin, destination)
                #return nx.bidirectional_dijkstra(subGraph, origin, destination)
            except:
                raise ValueError(
                    "No path was found with the given boundary conditions."
                )

        # If not, return shortest path
        else:
            return nx.dijkstra_path(graph, origin, destination)

#### 3.3. Create activity/activities

In [22]:
# initialise registry
registry = {}

In [23]:
# create a list of the sub processes for HAM311
sub_processes = [
    model.BasicActivity(
        env=my_env,
        name="basic activity",
        registry=registry,
        duration=0,
        additional_logs=[ham311]
    ),
    model.BasicActivity(
        env=my_env,
        name="basic activity",
        registry=registry,
        duration=0,
        additional_logs=[asia],
    )]
        
# create a 'sequential activity' that is made up of the 'sub_processes'
sequential_activity = model.SequentialActivity(
    env=my_env,
    name="Sequential activity of basic activities",
    registry=registry,
    sub_processes=sub_processes,
)

### 4. Register processes and run simpy

In [24]:
# initate the simpy processes defined in the 'while activity' and run simpy
model.register_processes([sequential_activity])
my_env.run()

AssertionError: 

### 5. Inspect Results

#### 5.1. Inspect logs
The method plot.get_log_dataframe returns the log of an activity in the form of a dataframe. By adding other activities in a list as the second argument, the Activity can be made more human readable.

In [ ]:
display(plot.get_log_dataframe(reporting_activity, [*sub_processes, parallel_activity, reporting_activity]))

#### 5.2. Visualize Gantt Chart

In [ ]:
plot.get_gantt_chart([while_activity])

In [ ]:
plot.get_gantt_chart([ham311, from_site, to_site],id_map=[while_activity])

In [ ]:
fig = plot.get_step_chart([ham311, from_site, to_site])

### Production of Trailing Suction Hopper Dredger (TSHD) and Water Injection Dredgers (WID)

#### Production of TSHD

Production rate of trailing suction hopper dredgers (TSHDs) is a metric for calculating the amount of dredged sediments during a single dredging cycle. The higher the production rate of TSHDs, the less time will be spent for conducting the project. The production of TSHDs varies over time based on the sailing distance to discharging area, capacity of hopper, operational hours, and sediment characteristics.

In this notebook, basic parameters related to vessel characteristics and the time required for loading, sailing (full and empty), and unloading are considered.

#### Production of WID

Production rate of water injection dredgers (WIDs) is a meteric for calculating the amount of sediments washed out from the water column during a single dredging cycle. The production of WIDs is mainly dependent on the penetration depth of the jet bar and the trailing velocity of vessel. Also, the width of jet bar is a determining factor in measuring the production. The overlap between two adjoining dredging tracks is subtracted from jet bar width to estimate the correct rate of production.

In this notebook, basic parameters related to vessel characteristics are developed for calculating the production rate.

It is assumed that the sediment to be dredged in the system mainly contains soft sediments which is the case for maintenance dredging. Also, the project is perfomed continuously in time. Therefore, the dredging cycle is continouos and the production rate is calculated for each dredging cycle which gives us an insight into comparing the production in different cycles.

In [15]:
class Production:
    """
    
    Mixin class: These parameters are used to estimate the production of vessels (TSHD and WID)
    type: can contain info on vessel type (TSHD or WID) [-]
    
    for TSHDs we have:
    h: hopper capacity [m^3]
    f_e: proportion of hopper that is filled [m^3]
    b: bulking factor [-]
    t_l: time required for loading [h]
    t_se: time required for sailing empty [h]
    t_sf: time required for sailing full [h]
    t_d: time required for discharging [h]
    t_tot: total time required for a single dredging cycle [h]
    sg_s: specific gravity of slurry [m/s^2]
    sg_so: specific gravity of solid [m/s^2]
    sg_f: specific gravity of fluid [m/s^2]
    dist: distance from dredging site to discharging site [km]
    l_d: length of the dredging area [km]
    v_se: velocity of vessel when sailing empty [knots]
    v_sf: velocity of vessel when sailing full [knots]
    prod_TSHD: total production of TSHD [m^3/h]
    
    for WIDs we have:
    v_t: trailing velocity [m/s]
    j_w: jet bar width [m]
    o: overlap between two adjoining dredging tracks [m]
    pen_d: penetration depth [m]
    prod_WID: total production of WID [m^3/s]
    s_u: undrained shear strength (kPa)
    den_w: water density [Kg/m^3]
    
    Add information on possible restrictions to the vessels, i.e. height, width, etc.
    """
    
    def __init__(
        self,
        prod_TSHD,
        prod_WID,
        f_e,
        pen_d,
        dist,
        t_l,
        t_se,
        t_sf,
        t_d,
        v_t = 8,
        j_w = 11,
        o = 1,
        sg_s = 1.2,
        sg_so = 2.2,
        sg_f = 0.35,
        l_d = 2_000,
        s_u = 3,
        den_w = 0.95,
        h = 10_000,
        b = 1.4,
        d_nozzle = 11,
        p_nozzle = 70,
        *args, 
        **kwargs
        ):
        super().__init__(*args, **kwargs)

        
        """Initialization"""
        self.type_vessel = type_vessel
        self.h = h
        self.f_e = f_e 
        self.b = b 
        self.t_l = t_l
        self.t_t = t_t
        self.t_se = t_se
        self.t_sf = t_sf
        self.t_d = t_d
        self.t_180 = t_180
        self.t_tot = t_tot
        self.n_t = n_t
        self.v_t = v_t
        self.j_w = j_w
        self.o = o
        self.pen_d = p_d
        self.sg_s = sg_s
        self.sg_so = sg_so
        self.sg_f = sg_f
        self.q = q
        self.dist = dist
        self.l_d = l_d
        self.v_se = v_se
        self.v_sf = v_sf
        self.prod_TSHD = prod_TSHD
        self.prod_WID = prod_WID
        self.s_u = s_u
        self.den_w = den_w

#### Calculating Trailing Suction Hopper Dredgers production parameters

In [28]:
##The proportion of the hopper that is filled f_e is considered equal to the ratio of solids to the amount of slurry

"""The proportion of hopper that is filled and the dredging cycle time is calculated according to Bray et al. (1997)"""

def calculate_hopper_proportion(self):
    
    self.f_e = (self.sg_s - self.sg_f) / (self.sg_s - self.sg_f)

##Total dredging cycle time comprises of the time required for loading, sailing (empy, full), and discharging 

def calculate_production_TSHD(self):
    
    self.calculate_hopper_proportion()
    self.calculate_cycle_time_TSHD()
    
    self.prod_TSHD = (self.h * self.f_e) / (self.t_l + self.t_se + self.t_sf + self.t_d)
    
    print('The production of TSHD is', self.prod_TSHD, 'm^3/s')

#### Calculating Water Injection Dredgers production parameters

In [29]:
"""Penetration depth and production of WID is calculated based on (WID Handbook, 2020) and (Machin and Allan, 2011)"""

def calculate_penetration_depth(self):
    
    self.pen_d = 24 * self.d_nozzle * (self.p_nozzle ^(1/2)) / (self.s_u ^(1/2))

def calculate_production_WID(self): 
    
    self.prod_WID = self.v_t * (self.j_w - self.o) * self.pen_d
    
    print("The production of WID is", self.prod_WID, 'm^3/s')